In [1]:
from scipy.integrate import odeint
import scipy.stats as ss
import math 
import numpy as np
import matplotlib.pyplot as plt
import theano
%matplotlib inline

theano.config.compute_test_value = 'raise'
theano.config.exception_verbosity= 'high'
THEANO_FLAGS='floatX=float64,optimizer=fast_compile'

#  Fitzhugh Nagumo model in oscillatory mode
In this example we will use PyMC3 to infer the parameters of the Fitzhugh-Nagumo oscillator. It is a simple model that 
generates action potentials. The model is describes as:

\begin{align}
\frac{dV}{dt}&=(V - \frac{V^3}{3} + R)c\\
\frac{dR}{dt}&=\frac{-(V-a+bR)}{c},
\end{align}
where $a$, $b$, $c$ are the model parameters.

We start by defining our ODE solver. In this case we will be using scipy odeint. However, note that we can also wrapp any other solver written in say C++. The point I am trying to drive home is that we are free to call up any model solver as long as it is available in python.

In [ ]:
def model_deriv(y, t, param):
    #Define parameters
    a,b,c=param
    
    #define states
    V,R=y
    #define derivatives
    dV_dt=(V-((V**3)/3) +R)*c
    dR_dt=-(V+b*R)/c
    return dV_dt,dR_dt
    
def model_sol(param):
    
    y0 = [-1,1]
    time = np.linspace(0, 20, 250)
    solution = odeint(model_deriv, y0, time, args=(param,))
    return np.array(solution)

#  Generate an artificial dataset

Having defined our model solver and forward solution, we can generate some synthetic data. We solve the model between [0,20] with 200 equispaced time points and add Gaussian noise with $\sigma=0.5$.

In [ ]:
times = np.linspace(0, 20, 250)
sigma=0.5
sol=model_sol([0.2,0.2,3.])
Y=sol+np.random.randn(250,2)*sigma
plt.figure(figsize=(15, 7.5))
plt.plot(times, sol, '-', color='#ff7f0e', lw=4, label='original values')
plt.plot(times, Y, 'o', color='#7f7f7f', ms=6.5, label='data points')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Value')

#  Define a black-box op using Theano @as_op

This is perhaps the most important pat of this example. Here we are defining a custom Theano as_op. Note that we are 
wrapping up the forward solution of our model as an as_op. Thus we tell Theano that we have three parameters each 
being a Theano scalar. The output then is a Theano matrix whose colums are the state vectors.

In [ ]:
import theano.tensor as tt
from theano.compile.ops import as_op

#@as_op(itypes=[tt.dscalar,tt.dscalar,tt.dscalar], otypes=[tt.dmatrix])
@as_op(itypes=[tt.dvector], otypes=[tt.dmatrix])
def Fitzhugh_ode_sol(param):
    
    #param=[param1,param2,param3]
    sol=model_sol(param)
    
    return sol


#  The probabilistic model: A lego approach

Next we define the probabilistic program. The likelihood is a multivariate Normal distribution. The priors are set as follows:
$a\sim \mathcal{\Gamma}(2,1)$, $b\sim \mathcal{N}(0,1)$ and $c\sim \mathcal{U}(0,10)$. In this example we will be estimating the noise standard deviation and thus we put the following prior: $\sigma\sim \mathcal{N}(0,1)$. 

In [ ]:
import pymc3 as pm
import theano.tensor as tt
from pymc3.theanof import floatX

class Unit(pm.Continuous):
    def __init__(self, lower, upper, *args, **kwargs):
        super(Unit, self).__init__(*args, **kwargs)
        self.lower = lower = tt.as_tensor_variable(floatX(lower))
        self.upper = upper = tt.as_tensor_variable(floatX(upper))
        self.mean = (upper + lower) / 2.
        self.median = self.mean

    def random(self, point=None, size=None, repeat=None):
        r=pm.Uniform.dist(lower=self.lower,upper=self.upper)
        return r.random(size=size)
    def logp(self, value):
        lower = self.lower
        upper = self.upper
        lt = tt.as_tensor_variable(floatX(2.9))
        gt = tt.as_tensor_variable(floatX(2.05))
        r=pm.Uniform.dist(lower=lower,upper=upper)
        lp = r.logp(value)
        outside = tt.as_tensor_variable(floatX(-np.inf))
        cond = tt.and_(tt.gt(value,gt),tt.lt(value,lt))
        return tt.switch(cond, lp, outside)

In [ ]:
@as_op(itypes=[tt.dscalar,tt.dscalar,tt.dscalar], otypes=[tt.dmatrix])
def randsample(lower,upper,samples):
    print(ss.uniform(lower,upper).rvs(int(samples)).shape)
    s1=ss.uniform(lower,upper).rvs(int(samples))
    s2=ss.uniform(lower,upper).rvs(int(samples))
    s3=ss.uniform(lower,upper).rvs(int(samples))
    sam=np.array([s1,s2,s3])
    return sam.T

@as_op(itypes=[tt.dscalar,tt.dscalar,tt.dvector], otypes=[tt.dvector])
def log_p(lower,upper,value):
    
    #lp = np.sum(ss.uniform(lower,upper).logpdf(value))
    return ss.uniform(lower,upper).logpdf(value)

class UnitMulti(pm.Continuous):
    def __init__(self, lower, upper, *args, **kwargs):
        super(UnitMulti, self).__init__(*args, **kwargs)
        self.lower = lower = tt.as_tensor_variable(floatX(lower))
        self.upper = upper = tt.as_tensor_variable(floatX(upper))
        self.mean = (upper + lower) / 2.
        self.median = self.mean

    def random(self, point=None, size=None, repeat=None):
        #r=pm.Uniform.dist(lower=self.lower,upper=self.upper)
       
        #return r.random(size=size)
        
        
        
        samples=tt.as_tensor_variable(floatX(size))
        r=randsample(self.lower,self.upper,samples)
        
        return r
    def logp(self, value):

        return log_p(self.lower,self.upper,value)
    
    

In [ ]:
from pymc3.step_methods import smc
from tempfile import mkdtemp
from scipy import optimize

theano.config.floatX = 'float64'
basic_model = pm.Model()
n_steps = 20 # Number of MCMC steps for each particle
n_chains = 100 # Number of particles (chains)

with basic_model:

    # Priors for unknown model parameters
    
    
    #a = pm.Gamma('a', alpha=2,beta=1, transform=None)
    #b = pm.Normal('b', mu=0, sd=1)
    #c = UnitMulti('c', lower=0, upper=10, transform=None, shape=3)
    c = pm.Uniform('c', lower=0, upper=10, transform=None, shape=3)
    sigma = pm.HalfNormal('sigma', sd=1)


    # Forward model
    
    mu = Fitzhugh_ode_sol(c)
    
    # Likelihood (sampling distribution) of observations
    cov=np.eye(2)*sigma**2
    Y_obs = pm.MvNormal('Y_obs', mu=mu, cov=cov, observed=Y)

    
    test_folder = mkdtemp(prefix='SMC_TEST')
    
     # Initial points for each of the chains
    #startsmc=[{'a':np.random.uniform(0,15,1),'b':np.random.uniform(0,15,1),
    #        'c':np.random.uniform(2.5,3.5,1),'sigma':np.random.uniform(0,2)} for _ in range(n_chains)]
    startsmc=[{'c':np.random.uniform(0.0,10.0,3),'sigma':np.random.uniform(0,2)} for _ in range(n_chains)]
    
    # Call SMC sampler
    db = pm.backends.ndarray.NDArray('test')
    trace = smc.sample_smc(n_steps=n_steps,n_chains=n_chains,progressbar=True,
                          homepath=test_folder,stage=0,tune_interval=10,random_seed=20,trace=db)

In [ ]:
pm.backends.text.dump('test',trace)
ltrace=pm.backends.text.load('test',basic_model)

In [ ]:
df=pm.trace_to_dataframe(ltrace)
df.head()
np.log(basic_model.marginal_likelihood)


In [ ]:
pooled_waic = pm.waic(trace, basic_model)

In [ ]:
np.array(pooled_waic)

In [ ]:
basic_model.logpt({'c':np.random.uniform(0.0,10.0,3),'sigma':np.random.uniform(0.2)})

In [ ]:
smc.proposal_dists.keys()

#  Visualise the posterior densities

We inspect first the traceplots to see whether the chains are mixing well enough.

In [ ]:
pm.traceplot(trace)

Nice visualisations support in-built

In [ ]:
pm.plot_posterior(trace,color='LightSeaGreen')

#  PyMC3 backend as Dataframes

We can use PyMC3 backend object that represents the chain(s) and use some pandas magic

In [ ]:
import pandas as pd

results=[pm.df_summary(trace, ['a']),pm.df_summary(trace, ['b']),pm.df_summary(trace, ['c'])\
        ,pm.df_summary(trace, ['sigma'])]
results=pd.concat(results)

results['True values'] = pd.Series(np.array([0.2,0.2,3.0,0.5]), index=results.index)
results

#  Posterior predictive plots

In [ ]:
params=np.array([trace.get_values('a'),trace.get_values('b'),trace.get_values('c')]).T
params.shape

In [ ]:
new_values = []
for ind in range(len(params)):
    ppc_sol=model_sol(params[ind])
    new_values.append(ppc_sol)
new_values = np.array(new_values)
mean_values = np.mean(new_values, axis=0)
new_values.shape
plt.figure(figsize=(15, 7.5))
plt.plot(times, new_values[0], color='yellow', alpha=0.05, label='inferred concentration')
for v in new_values[1:]:
    plt.plot(times, v, color='yellow', alpha=0.05)
plt.plot(times, mean_values, color='black', lw=4, label='mean of inferred')
plt.plot(times, sol, '--', color='#ff7f0e', lw=4, label='original concentration')
plt.plot(times, Y, 'o', color='#7f7f7f', ms=6.5, label='data points')
plt.legend()
plt.xlabel('Time')
plt.ylabel('concentration')


#  Remember we get the marginal likelihood for free

In [ ]:
def beta_binom(prior, y):
    """
    Compute the marginal likelihood, analytically, for a beta-binomial model.

    prior : tuple
        tuple of alpha and beta parameter for the prior (beta distribution)
    y : array
        array with "1" and "0" corresponding to the success and fails respectively
    """
    alpha, beta = prior
    h = np.sum(y)
    n = len(y)
    p_y = np.exp(betaln(alpha + h, beta+n-h) - betaln(alpha, beta))
    return p_y

In [2]:
y = np.repeat([1, 0], [50, 50])  # 50 "heads" and 50 "tails"
priors = ((1, 1), (30, 30))

In [3]:
from pymc3.step_methods import smc
from tempfile import mkdtemp
from scipy import optimize
import pymc3 as pm
theano.config.floatX = 'float64'


n_chains = 1000

models = []
traces = []
for alpha, beta in priors:
    test_folder = mkdtemp(prefix='SMC_TEST')
    with pm.Model() as model:
        a = pm.Beta('a', alpha, beta, transform=None)
        yl = pm.Bernoulli('yl', a, observed=y)
        trace = smc.sample_smc(samples=5000,n_chains=n_chains,progressbar=True,
                        homepath=test_folder,stage=0,random_seed=21)
        models.append(model)
        traces.append(trace)

/home/sanosh/pymc3/pymc3/step_methods/smc.py:491: UserWarning: Warning: SMC is an experimental step method, and not yet recommended for use in PyMC3!
  warnings.warn(EXPERIMENTAL_WARNING)
/home/sanosh/pymc3/pymc3/step_methods/smc.py:118: UserWarning: Warning: SMC is an experimental step method, and not yet recommended for use in PyMC3!
  warnings.warn(EXPERIMENTAL_WARNING)
100%|██████████| 1000/1000 [00:00<00:00, 1483.27it/s]


In [4]:
BF_smc = models[1].marginal_likelihood / models[0].marginal_likelihood
print(round(BF_smc))

5.0


In [5]:
print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.3
